In [70]:
import pandas as pd
import numpy as np
import time
import math
from python_pkg import python_udf as udf

In [2]:
#import the data
df = pd.read_csv('C:/Users/Joe/Projects/fantasy_football/playground/df.csv')

In [3]:
df.head()

,pk,gid,seas,wk,player,fname,lname,full_name,team,pos1,...,humd,ou,sprv,ptsv,ptsh,udog,gen_cond,f_pts,udog_binary,def_team
0,3999&AA-0025,3999,2015,1,AA-0025,Ameer,Abdullah,Ameer Abdullah,DET,RB,...,47.0,46.0,3.0,28,33,DET,fair_cond,18.400000,True,SD
1,4010&AA-0025,4010,2015,2,AA-0025,Ameer,Abdullah,Ameer Abdullah,DET,RB,...,NaN,43.5,2.0,16,26,DET,fair_cond,4.200000,True,MIN
2,4036&AA-0025,4036,2015,3,AA-0025,Ameer,Abdullah,Ameer Abdullah,DET,RB,...,45.0,44.5,-3.0,24,12,DET,indoor_cond,10.200000,True,DEN
3,4052&AA-0025,4052,2015,4,AA-0025,Ameer,Abdullah,Ameer Abdullah,DET,RB,...,41.0,43.0,10.0,10,13,DET,fair_cond,5.942857,True,SEA
4,4062&AA-0025,4062,2015,5,AA-0025,Ameer,Abdullah,Ameer Abdullah,DET,RB,...,45.0,44.5,-3.0,42,17,DET,indoor_cond,1.671429,True,ARI


In [37]:
#RB only
df_rb = df[df['pos1']=='RB']

In [68]:
#recent_ra grouped
df_grouped_total = df_rb.groupby(by=['team', 'seas', 'wk', 'pk'], as_index=True).agg({'recent_ra': 'sum'})

In [74]:
#calc the percent of group total
start = time.time()
df_pct = df_grouped_total.groupby(level=[0,1,2]).apply(lambda x: x/x.sum())
end = time.time()

In [75]:
print('Calc time: {:.2f}'.format(end-start))

Calc time: 19.00


In [117]:
df_pct_full = df_pct.reset_index()
df_pct_full.head()

,team,seas,wk,pk,recent_ra
0,ARI,2000,1,10&JM-0700,NaN
1,ARI,2000,1,10&MP-1300,NaN
2,ARI,2000,1,10&TJ-2700,NaN
3,ARI,2000,2,29&MP-1300,0.25
4,ARI,2000,2,29&TJ-2700,0.75


In [140]:
max_pct = df_pct_full.groupby(by=['team', 'seas', 'wk']).agg({'recent_ra': 'max'}).reset_index()
max_pct.shape

(9092, 4)

In [142]:
rb_committee = pd.Series(np.where(max_pct['recent_ra'] < 0.60, 1, 0), name='committee')

In [151]:
rb_pct_splits = [-np.inf, 0.33, 0.67, 0.9, np.inf]
rb_split_labels = ['spell', 'committee', 'primary', 'star']
rb_category = pd.cut(df_pct_full['recent_ra'], bins=rb_pct_splits, right=False, labels=rb_split_labels).rename('rb_type')

In [152]:
category_result = pd.concat([df_pct_full, rb_category], axis=1)

In [155]:
def rb_committee(rb_data, pos_grid, bin_threshold=None, cat_splits=None, cat_labels=None, binary=True):
    """Create feature to determine if RB is part of a commitee\n
            Returns a dataframe to merge to original data"""
    #make sure data types are correct
    assert isinstance(pos_grid, dict)
    assert isinstance(binary, bool)
    
    #get the column to calculate % of work from
    stat_col = pos_grid['RB']
    
    #group to get the total stat_col for the given grouping
    df_grouped_total = rb_data.groupby(by=['team', 'seas', 'wk', 'pk'], as_index=True).agg({stat_col: 'sum'})
    #calc the % of total for each player
    df_pct = df_grouped_total.groupby(level=[0,1,2]).apply(lambda x: x/x.sum())
    #reset index for next round of groupby
    df_pct_full = df_pct.reset_index()
    
    #calc the maximum percent for the group; used to determine if player is star back or not
    max_pct = df_pct_full.groupby(by=['team', 'seas', 'wk']).agg({stat_col: 'max'}).reset_index()
    #binary feature if RB is part of a committee or not
    rb_binary = pd.Series(np.where(max_pct[stat_col] < bin_threshold, 1, 0), name='committee')
    
    #categorical for type of RB workload (star, committee, spell)
    rb_category = pd.cut(df_pct_full[stat_col], bins=cat_splits, right=False, labels=cat_labels).rename('rb_type')
    
    #merge columns to rb_data; to be used for merging to normal dataframe
    binary_result = pd.concat([max_pct, rb_binary], axis=1)
    category_result = pd.concat([df_pct_full, rb_category], axis=1)
    
    #return the binary or categorical
    if binary == True:
        return binary_result
    else:
        return category_result

In [157]:
depth_chart_grid = {'QB': 'recent_pa',
                        'RB': 'recent_ra',
                        'WR': 'recent_trg',
                        'TE': 'recent_trg'}

In [167]:
output = rb_committee(df[df['pos1']=='RB'], depth_chart_grid, bin_threshold=0.6,
                         cat_splits=rb_pct_splits, cat_labels=rb_split_labels, binary=False)

In [ ]:
output